## Summary:

This Python program along with 2f will be used to gather and augment the following new columns into the dataframe created for regression earlier i.e. RW_authors_w_confounders_filteredSample_postNHB.csv.

The new columns we are interested in are:
1. First affiliation rank -- this was already fixed
2. First year (this will vary a bit now because Bedoor's using MAG to compute the publication year, and some  authors might now have attrition year before their first publication year. In any case, we need to correct for this now).
3. Attrition Year (this is as computed by Bedoor) -- We need to drop the old one and rename the new
4. Attrition Class (this will be based on the logic discussed with Bedoor)
    
    - Attrition class is 1 (attrited) if year of attrition minus retraction year is 0 or -1
    - Attrition class is 0 (not attrited) if it is positive
    - Attrtion class is 0 (not attrited) if year of Attrition is -1
    - Attrition class is -1 (gray) if it is less than -1
    
5. We will update academic age as for about 300 instances it may be have been off by +-1 due to discrepancies in MAG and RW's publication years.

6. We will then correct/update the author order in the retracted paper as we realized that for repeated offenders, this was messed up for some reason.

7. We will also add "retractor" field as that was missing in the dataframe.

We will then do some sensibility checks.

In [97]:
import pandas as pd
import os
from config_reader import read_config

In [99]:
# Reading paths
paths = read_config()
# Path to where we will save our processed files
OUTDIR = paths['PROCESSED_FOLDER_LOCAL']

In [100]:
df = pd.read_csv(OUTDIR+"RW_authors_w_confounders_filteredSample_postNHB_BedoorsCorrections.csv")
df.head(1)

/var/folders/bf/_6xjm2bn2_n17hfhskyq1pmw0000gp/T/ipykernel_30356/2978764420.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(OUTDIR+"RW_authors_w_confounders_filteredSample_postNHB_BedoorsCorrections.csv")


,MAGAID,MAGAuthorName,Record ID,MAGPID,RetractionYear,nRetracted,dateobject,JID,CSID,JournalName,ConferenceSeriesName,ReasonPropagatedMajorityOfMajority,NumAuthorsInRetractedPaper,GenderizeGender,GenderizeConfidence,MAGAuthorOrder,FirstPubMAGPID,FirstPubYear,AcademicAgeAtRetraction,cumPapersYear,MAGCumPapersAtRetraction,cumCitationsYear,MAGCumCitationsAtRetraction,cumCollaboratorsYear,MAGCumCollaboratorsAtRetraction,AffID,MAGRetractionYearAffRank,AffYear,MAGrootFID,MAGrootFIDMaxPercent,YearOfAttrition_x,YearsActive,OriginalPaperDate,OriginalPaperYear,AttritedClass,AttritedClassRobust,MAGJCName,MAGJournalType,SJRScoreRetractedPaperYear,SJRQuartileRetractedPaperYear,DemiDecade,MAGAIDRankTypeInRetractedPaper,MAGFieldName,Field_art,Field_biology,Field_business,Field_chemistry,Field_computerscience,Field_economics,Field_engineering,Field_environmentalscience,Field_geography,Field_geology,Field_history,Field_materialsscience,Field_mathematics,Field_medicine,Field_philosophy,Field_physics,Field_politicalscience,Field_psychology,Field_sociology,MAGRetractionYearAffRankOrdinal,LogMAGCumCitationsAtRetraction,LogMAGCumCollaboratorsAtRetraction,BlogMentionsAtRetraction,F1000MentionsAtRetraction,FacebookMentionsAtRetraction,Google+MentionsAtRetraction,LinkedInMentionsAtRetraction,NewsMentionsAtRetraction,PatentMentionsAtRetraction,Peer reviewMentionsAtRetraction,PinterestMentionsAtRetraction,PolicyMentionsAtRetraction,Q&AMentionsAtRetraction,RedditMentionsAtRetraction,TwitterMentionsAtRetraction,VideoMentionsAtRetraction,WeiboMentionsAtRetraction,WikipediaMentionsAtRetraction,TotalMentionsAtRetraction,AggregateSocialMediaMentionsAtRetraction,AggregateNewsMediaMentionsAtRetraction,AggregateBlogsMentionsAtRetraction,AggregateKnowledgeRepositoriesMentionsAtRetraction,BlogMentionsPreRetraction,F1000MentionsPreRetraction,FacebookMentionsPreRetraction,Google+MentionsPreRetraction,LinkedInMentionsPreRetraction,NewsMentionsPreRetraction,PatentMentionsPreRetraction,Peer reviewMentionsPreRetraction,PinterestMentionsPreRetraction,PolicyMentionsPreRetraction,Q&AMentionsPreRetraction,RedditMentionsPreRetraction,TwitterMentionsPreRetraction,VideoMentionsPreRetraction,WeiboMentionsPreRetraction,WikipediaMentionsPreRetraction,TotalMentionsPreRetraction,AggregateSocialMediaMentionsPreRetraction,AggregateNewsMediaMentionsPreRetraction,AggregateBlogsMentionsPreRetraction,AggregateKnowledgeRepositoriesMentionsPreRetraction,BlogMentionsPostRetraction,F1000MentionsPostRetraction,FacebookMentionsPostRetraction,Google+MentionsPostRetraction,LinkedInMentionsPostRetraction,NewsMentionsPostRetraction,PatentMentionsPostRetraction,Peer reviewMentionsPostRetraction,PinterestMentionsPostRetraction,PolicyMentionsPostRetraction,Q&AMentionsPostRetraction,RedditMentionsPostRetraction,TwitterMentionsPostRetraction,VideoMentionsPostRetraction,WeiboMentionsPostRetraction,WikipediaMentionsPostRetraction,TotalMentionsPostRetraction,AggregateSocialMediaMentionsPostRetraction,AggregateNewsMediaMentionsPostRetraction,AggregateBlogsMentionsPostRetraction,AggregateKnowledgeRepositoriesMentionsPostRetraction,AltmetricScoreAtRetraction,AltmetricScorePreRetraction,AltmetricScorePostRetraction,FirstAffID,FirstAffRank,FirstAffRankCorrected,YearOfAttrition_y
0,2127983451,william r jacobs,2343,1790891469,1997.0,1,1994-07-01,133274750.0,NaN,journal of bacteriology,NaN,mistake,5.0,male,0.99,5,2077672427,1976.0,21.0,1997.0,109,1997.0,2441.0,1997.0,266.0,129975664.0,1001-,1997.0,86803240.0,0.313531,1978.0,-19.0,1994-07-01,1994.0,-1,-1,journal of bacteriology,journal,NaN,NaN,1996-2000,First or Last or Only Author,biology,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89.0,7.800573,5.587249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,3.0,6.0,204465549.0,22.0,NaN,-1.0


In [3]:
df.columns.tolist()

['MAGAID',
 'MAGAuthorName',
 'Record ID',
 'MAGPID',
 'RetractionYear',
 'nRetracted',
 'dateobject',
 'JID',
 'CSID',
 'JournalName',
 'ConferenceSeriesName',
 'ReasonPropagatedMajorityOfMajority',
 'NumAuthorsInRetractedPaper',
 'GenderizeGender',
 'GenderizeConfidence',
 'MAGAuthorOrder',
 'FirstPubMAGPID',
 'FirstPubYear',
 'AcademicAgeAtRetraction',
 'cumPapersYear',
 'MAGCumPapersAtRetraction',
 'cumCitationsYear',
 'MAGCumCitationsAtRetraction',
 'cumCollaboratorsYear',
 'MAGCumCollaboratorsAtRetraction',
 'AffID',
 'MAGRetractionYearAffRank',
 'AffYear',
 'MAGrootFID',
 'MAGrootFIDMaxPercent',
 'YearOfAttrition_x',
 'YearsActive',
 'OriginalPaperDate',
 'OriginalPaperYear',
 'AttritedClass',
 'AttritedClassRobust',
 'MAGJCName',
 'MAGJournalType',
 'SJRScoreRetractedPaperYear',
 'SJRQuartileRetractedPaperYear',
 'DemiDecade',
 'MAGAIDRankTypeInRetractedPaper',
 'MAGFieldName',
 'Field_art',
 'Field_biology',
 'Field_business',
 'Field_chemistry',
 'Field_computerscience',
 'Fi

In [4]:
# Checking number of authors

df['MAGAID'].nunique(), df[df['nRetracted'] == 1]['MAGAID'].nunique()

(15981, 14579)

In [5]:
# Checking that first affiliation ranks
df[['MAGAID','FirstAffRankCorrected']].drop_duplicates()['FirstAffRankCorrected'].value_counts(dropna=False)

FirstAffRankCorrected
1500.0    6209
NaN       3315
250.0     1264
125.0     1054
175.0      995
350.0      969
450.0      702
750.0      563
550.0      559
650.0      443
850.0      373
950.0      371
Name: count, dtype: int64

In [6]:
# Why are we seeing NaNs?

df[df['FirstAffRankCorrected'].isna()][['MAGAID','FirstAffRank']].drop_duplicates()
# Seems like it is because I did not account for when authors have <=100 as affiliation. Lets fix that

df['FirstAffRankCorrected'] = df['FirstAffRankCorrected'].fillna(df['FirstAffRank'])

df[['MAGAID','FirstAffRankCorrected']].drop_duplicates()['FirstAffRankCorrected'].value_counts(dropna=False)
# An author can have multiple affiliation ranks, so I haven't filtered them just yet.

FirstAffRankCorrected
1500.0    6209
250.0     1264
125.0     1054
175.0      995
350.0      969
          ... 
37.0         9
60.0         8
98.0         8
48.0         6
44.0         1
Name: count, Length: 88, dtype: int64

In [7]:
# Let us fix first year now

df[df['FirstPubYear'].gt(df['YearOfAttrition_y']) & df['YearOfAttrition_y'].ne(-1)]['MAGAID'].nunique()

# These are authors that need to be fixed


146

In [8]:
df.loc[(df['FirstPubYear'] > df['YearOfAttrition_y']) & (df['YearOfAttrition_y'] != -1), 'FirstPubYear'] = df['YearOfAttrition_y']


df[df['FirstPubYear'].gt(df['YearOfAttrition_y']) & df['YearOfAttrition_y'].ne(-1)]['MAGAID'].nunique()

0

In [9]:
# Let us now drop and rename YearOfAttrition


df = df.drop(columns=['YearOfAttrition_x'])\
        .rename(columns={'YearOfAttrition_y':'YearOfAttrition'})

df.head(1)

,MAGAID,MAGAuthorName,Record ID,MAGPID,RetractionYear,nRetracted,dateobject,JID,CSID,JournalName,...,AggregateNewsMediaMentionsPostRetraction,AggregateBlogsMentionsPostRetraction,AggregateKnowledgeRepositoriesMentionsPostRetraction,AltmetricScoreAtRetraction,AltmetricScorePreRetraction,AltmetricScorePostRetraction,FirstAffID,FirstAffRank,FirstAffRankCorrected,YearOfAttrition
0,2127983451,william r jacobs,2343,1790891469,1997.0,1,1994-07-01,133274750.0,NaN,journal of bacteriology,...,0.0,0.0,2.0,0.0,3.0,6.0,204465549.0,22.0,22.0,-1.0


In [10]:
df['AttritedClass']

0       -1
1       -1
2       -1
3       -1
4       -1
        ..
38459    1
38460    1
38461    0
38462    1
38463    1
Name: AttritedClass, Length: 38464, dtype: int64

In [11]:
# Let us now compute attrition class

df['AttritedClass'] = 0

df.loc[(df.YearOfAttrition-df.RetractionYear)==0, 'AttritedClass'] = 1
df.loc[(df.YearOfAttrition-df.RetractionYear)==-1, 'AttritedClass'] = 1
df.loc[(df.YearOfAttrition-df.RetractionYear)<-1, 'AttritedClass'] = -1
df.loc[df.YearOfAttrition==-1, 'AttritedClass'] = 0



In [12]:
df[['MAGAID','AttritedClass']].drop_duplicates()['AttritedClass'].value_counts()

AttritedClass
 0    11335
 1     2618
-1     2028
Name: count, dtype: int64

In [13]:
"""
We went back and forth on 1 hot and n hot. And decided to do n-hot 
so I am commenting this. 
"""

# Finally, let us correct the fields.

# Removing all old Field columns

# Identify columns starting with "Field_"
# columns_to_drop = [col for col in df.columns if col.startswith('Field_')]

# print(columns_to_drop)

# # Drop these columns
# df = df.drop(columns=columns_to_drop).drop_duplicates()

# df.columns.tolist()

'\nWe went back and forth on 1 hot and n hot. And decided to do n-hot \nso I am commenting this. \n'

In [14]:
"""
We went back and forth on 1 hot and n hot. And decided to do n-hot 
so I am commenting this. 
"""


# # Now let us create new columns


# # Perform one-hot encoding
# df_encoded = pd.get_dummies(df['MAGFieldName'], prefix='Field', drop_first=True)

# # Rename columns: add 'Field_' prefix, convert to lowercase, replace spaces with underscores
# df_encoded.columns = ['Field_' + col.split('_')[-1].replace(' ', '').lower() for col in df_encoded.columns]


# df_encoded.columns.shape

'\nWe went back and forth on 1 hot and n hot. And decided to do n-hot \nso I am commenting this. \n'

In [15]:
# df_encoded

,Field_biology,Field_business,Field_chemistry,Field_computerscience,Field_economics,Field_engineering,Field_environmentalscience,Field_geography,Field_geology,Field_history,Field_materialsscience,Field_mathematics,Field_medicine,Field_philosophy,Field_physics,Field_politicalscience,Field_psychology,Field_sociology
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38459,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
38460,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
38461,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
38462,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


In [15]:
# Concatenate with the original dataframe if needed
df_final = df

# Display the final dataframe
df_final.columns.tolist()

['MAGAID',
 'MAGAuthorName',
 'Record ID',
 'MAGPID',
 'RetractionYear',
 'nRetracted',
 'dateobject',
 'JID',
 'CSID',
 'JournalName',
 'ConferenceSeriesName',
 'ReasonPropagatedMajorityOfMajority',
 'NumAuthorsInRetractedPaper',
 'GenderizeGender',
 'GenderizeConfidence',
 'MAGAuthorOrder',
 'FirstPubMAGPID',
 'FirstPubYear',
 'AcademicAgeAtRetraction',
 'cumPapersYear',
 'MAGCumPapersAtRetraction',
 'cumCitationsYear',
 'MAGCumCitationsAtRetraction',
 'cumCollaboratorsYear',
 'MAGCumCollaboratorsAtRetraction',
 'AffID',
 'MAGRetractionYearAffRank',
 'AffYear',
 'MAGrootFID',
 'MAGrootFIDMaxPercent',
 'YearsActive',
 'OriginalPaperDate',
 'OriginalPaperYear',
 'AttritedClass',
 'AttritedClassRobust',
 'MAGJCName',
 'MAGJournalType',
 'SJRScoreRetractedPaperYear',
 'SJRQuartileRetractedPaperYear',
 'DemiDecade',
 'MAGAIDRankTypeInRetractedPaper',
 'MAGFieldName',
 'Field_art',
 'Field_biology',
 'Field_business',
 'Field_chemistry',
 'Field_computerscience',
 'Field_economics',
 'Fiel

In [16]:
# Let us finally create a column for affiliation rank ordinal

def make_rank_ordinal(rank):
    rank_dict = {'101-150':125,
             '151-200':175,
             '201-300':250,
             '301-400':350,
             '401-500':450,
             '501-600':550,
             '601-700':650,
             '701-800':750,
             '801-900':850,
             '901-1000':950,
             '1001-':1500}
    
    flipped_dict = {value: key for key, value in rank_dict.items()}
    if rank <= 100:
        return str(int(rank))
    return flipped_dict.get(rank)

df_final['MAGFirstAffiliationRank'] = df_final['FirstAffRankCorrected'].apply(make_rank_ordinal)


"""
At this point there is one last thing that needs to be done i.e. 
for authors who have multiple affiliations, we need to only take their 
high-ranked affiliation only

An important note:
We are changing first affiliation rank but we are not changing the affiliation id. 
And that is OK and in fact desired. Same goes for retraction year aff id. 
This is because at the time of matching, aff id is only used for >1000 and so 
we must use all the ones we have to match on.
"""

df_affrank = df_final[['MAGAID','MAGFirstAffiliationRank']].\
                drop_duplicates().dropna().sort_values(by='MAGFirstAffiliationRank').\
                drop_duplicates(subset='MAGAID',keep='first')


# Now we shall remove the MAGRetractionYearAffRankOrdinal column and reinclude it 
df_final = df_final.\
        drop(columns=['MAGFirstAffiliationRank']).merge(df_affrank, on='MAGAID',how='left')

# Rename MAGFID

df_final = df_final.rename(columns={'FirstAffID':'MAGFirstAffID',
                                   'FirstPubYear':'MAGFirstPubYear',
                                   'AffID':'MAGRetractionYearAffID',
                                   'AffYear':'MAGRetractionYearAffYear',
                                   'cumPapersYear':'MAGCumPapersYearAtRetraction',
                                   'cumCitationsYear':'MAGCumCitationsYearAtRetraction',
                                   'cumCollaboratorsYear':'MAGCumCollaboratorsYearAtRetraction',
                                   'MAGAIDRankTypeInRetractedPaper':'MAGAuthorOrderCategorical'})

df_final[['MAGAID','MAGFirstAffiliationRank']].drop_duplicates(), df_final['MAGAID'].nunique()

(           MAGAID MAGFirstAffiliationRank
 0      2127983451                      22
 5      1986180616                 101-150
 8      2134970185                   1001-
 9      2600580187                      28
 11      257122240                   1001-
 ...           ...                     ...
 38386   384584067                   1001-
 38406  2776957463                 101-150
 38408  2115767897                 101-150
 38418   582542066                   1001-
 38425  2100934451                   1001-
 
 [15981 rows x 2 columns],
 15981)

## Corrections

In [17]:
"""
Now we are going to update the academic age as we updated the first pub year of some authors
"""


df_final['AcademicAgeNew'] = df_final['RetractionYear'] - df_final['MAGFirstPubYear']

df_final[df_final['AcademicAgeNew'] != df_final['AcademicAgeAtRetraction']]

,MAGAID,MAGAuthorName,Record ID,MAGPID,RetractionYear,nRetracted,dateobject,JID,CSID,JournalName,...,AggregateKnowledgeRepositoriesMentionsPostRetraction,AltmetricScoreAtRetraction,AltmetricScorePreRetraction,AltmetricScorePostRetraction,MAGFirstAffID,FirstAffRank,FirstAffRankCorrected,YearOfAttrition,MAGFirstAffiliationRank,AcademicAgeNew
1925,3153423935,richard j sawyer,1696,2067865435,2009.0,1,2002-04-01,1.240357e+08,NaN,anesthesiology,...,0.0,0.0,0.0,0.0,121934306.0,151.0,175.0,2001.0,151-200,8.0
5094,2174907077,sixi liu,1685,1581868038,2015.0,1,2007-04-01,1.925397e+08,NaN,clinical anatomy,...,0.0,0.0,0.0,0.0,24185976.0,151.0,175.0,2006.0,151-200,9.0
5114,2646014357,chaohua yao,7330,1983643770,2007.0,1,2007-07-01,5.380716e+07,NaN,journal of cellular biochemistry,...,0.0,0.0,0.0,0.0,46635041.0,NaN,1500.0,2006.0,1001-,1.0
5124,2375006231,s yi,2747,2026286254,2010.0,1,2007-08-23,1.284400e+08,NaN,oncogene,...,0.0,0.0,0.0,5.0,170203145.0,NaN,1500.0,2006.0,1001-,4.0
5126,2569629445,mikiko ono,318,2036630809,2008.0,1,2007-01-01,2.000711e+08,NaN,blood,...,0.0,0.0,0.0,0.0,107139324.0,901.0,950.0,2006.0,901-1000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38352,2042304441,keshav krishna,22069,2107213397,2015.0,1,2015-10-01,1.985739e+08,NaN,environmental earth sciences,...,0.0,0.0,0.0,0.0,66760702.0,NaN,1500.0,2014.0,1001-,1.0
38353,2320993163,rama mohan,22069,2107213397,2015.0,1,2015-10-01,1.985739e+08,NaN,environmental earth sciences,...,0.0,0.0,0.0,0.0,66760702.0,NaN,1500.0,2014.0,1001-,1.0
38416,2240430476,ian coldwell,5036,1566674077,2012.0,1,2010-04-01,6.632250e+07,NaN,sociologia ruralis,...,0.0,0.0,0.0,0.0,165779595.0,41.0,41.0,2009.0,41,3.0
38440,2754828967,r sowmeyan,23083,2413904130,2009.0,1,2008-04-01,1.450900e+08,NaN,journal of hazardous materials,...,1.0,0.0,0.0,3.5,207025910.0,NaN,1500.0,2007.0,1001-,2.0


In [18]:
df_final['AcademicAgeAtRetraction'] = df_final['AcademicAgeNew']

In [19]:
df_final = df_final.drop(columns=['AcademicAgeNew'])

In [20]:
df_final.columns.tolist()

['MAGAID',
 'MAGAuthorName',
 'Record ID',
 'MAGPID',
 'RetractionYear',
 'nRetracted',
 'dateobject',
 'JID',
 'CSID',
 'JournalName',
 'ConferenceSeriesName',
 'ReasonPropagatedMajorityOfMajority',
 'NumAuthorsInRetractedPaper',
 'GenderizeGender',
 'GenderizeConfidence',
 'MAGAuthorOrder',
 'FirstPubMAGPID',
 'MAGFirstPubYear',
 'AcademicAgeAtRetraction',
 'MAGCumPapersYearAtRetraction',
 'MAGCumPapersAtRetraction',
 'MAGCumCitationsYearAtRetraction',
 'MAGCumCitationsAtRetraction',
 'MAGCumCollaboratorsYearAtRetraction',
 'MAGCumCollaboratorsAtRetraction',
 'MAGRetractionYearAffID',
 'MAGRetractionYearAffRank',
 'MAGRetractionYearAffYear',
 'MAGrootFID',
 'MAGrootFIDMaxPercent',
 'YearsActive',
 'OriginalPaperDate',
 'OriginalPaperYear',
 'AttritedClass',
 'AttritedClassRobust',
 'MAGJCName',
 'MAGJournalType',
 'SJRScoreRetractedPaperYear',
 'SJRQuartileRetractedPaperYear',
 'DemiDecade',
 'MAGAIDRankTypeInRetractedPaper',
 'MAGFieldName',
 'Field_art',
 'Field_biology',
 'Field_b

In [43]:
# Now let us update author order column
# Why did this happen?
# Because I chose one retracted paper for each repeated offender
# But forgot to choose the relevant author order

df_author_pubhistories = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/Retracted_Authors_PubHistories.csv",
                                    usecols=['MAGAID','MAGPID','RetractedPaperMAGPID','MAGAuthorOrder',
                                            'Record ID'])\
                                    .drop_duplicates()

df_author_pubhistories_rel = df_author_pubhistories[df_author_pubhistories['MAGPID'] == df_author_pubhistories['RetractedPaperMAGPID']]
df_author_pubhistories_rel


,MAGPID,MAGAID,MAGAuthorOrder,Record ID,RetractedPaperMAGPID
13,2273790572,569306176,9,23311,2273790572
174,2273790572,2230134732,2,23311,2273790572
485,2273790572,2423554807,7,23311,2273790572
633,2273790572,2462990842,15,23311,2273790572
863,2273790572,2477911198,5,23311,2273790572
...,...,...,...,...,...
2841736,3168732224,3170455413,4,6759,3168732224
2841737,3168732224,3171097566,3,6759,3168732224
2841738,3168732224,3171812317,1,6759,3168732224
2841739,3168732224,3172678465,7,6759,3168732224


In [64]:
# Separating single and repeated offenders

df_final_single_offenders = df_final[df_final['nRetracted']==1]
df_final_repeated_offenders = df_final[df_final['nRetracted']!=1]

In [78]:
df_final_single_offenders['MAGAID'].nunique(), df_final_single_offenders['MAGPID'].nunique(), df_final_single_offenders['Record ID'].nunique()


(14579, 4578, 4578)

In [67]:
df_final_repeated_offenders['MAGAID'].nunique(), df_final_repeated_offenders['MAGPID'].nunique(), df_final_repeated_offenders['Record ID'].nunique()


(1402, 1105, 1111)

In [68]:
# Removing redundant column(s) before merging
df_author_pubhistories_rel = df_author_pubhistories_rel.drop(columns=['RetractedPaperMAGPID'])\
                                                        .drop_duplicates()
df_author_pubhistories_rel

,MAGPID,MAGAID,MAGAuthorOrder,Record ID
13,2273790572,569306176,9,23311
174,2273790572,2230134732,2,23311
485,2273790572,2423554807,7,23311
633,2273790572,2462990842,15,23311
863,2273790572,2477911198,5,23311
...,...,...,...,...
2841736,3168732224,3170455413,4,6759
2841737,3168732224,3171097566,3,6759
2841738,3168732224,3171812317,1,6759
2841739,3168732224,3172678465,7,6759


In [92]:
df_final_repeated_offenders_corrected = df_author_pubhistories_rel.merge(df_final_repeated_offenders, 
                                               on=['MAGPID','MAGAID','Record ID','MAGAuthorOrder'])

df_final_repeated_offenders_corrected['Record ID'].nunique()


1111

In [93]:
# Let us select only one paper per repeated offender
pd.set_option('display.max_columns', None)

df_final_repeated_offenders_corrected['MinDate'] = \
                        df_final_repeated_offenders_corrected.groupby(['MAGAID'])['OriginalPaperDate']\
                        .transform('min')

df_final_repeated_offenders_corrected = df_final_repeated_offenders_corrected[df_final_repeated_offenders_corrected['MinDate']==\
                                         df_final_repeated_offenders_corrected['OriginalPaperDate']]

df_final_repeated_offenders_corrected = df_final_repeated_offenders_corrected.drop(columns=['MinDate'])


In [96]:
print(df_final_repeated_offenders_corrected['MAGAID'].nunique(),
      df_final_repeated_offenders_corrected['MAGPID'].nunique(),
      df_final_repeated_offenders_corrected['Record ID'].nunique())

1402 703 705


In [106]:
# Merging the two dataframes

df_final_new = pd.concat([df_final_single_offenders, df_final_repeated_offenders_corrected])
df_final_new

,MAGAID,MAGAuthorName,Record ID,MAGPID,RetractionYear,nRetracted,dateobject,JID,CSID,JournalName,ConferenceSeriesName,ReasonPropagatedMajorityOfMajority,NumAuthorsInRetractedPaper,GenderizeGender,GenderizeConfidence,MAGAuthorOrder,FirstPubMAGPID,MAGFirstPubYear,AcademicAgeAtRetraction,MAGCumPapersYearAtRetraction,MAGCumPapersAtRetraction,MAGCumCitationsYearAtRetraction,MAGCumCitationsAtRetraction,MAGCumCollaboratorsYearAtRetraction,MAGCumCollaboratorsAtRetraction,MAGRetractionYearAffID,MAGRetractionYearAffRank,MAGRetractionYearAffYear,MAGrootFID,MAGrootFIDMaxPercent,YearsActive,OriginalPaperDate,OriginalPaperYear,AttritedClass,AttritedClassRobust,MAGJCName,MAGJournalType,SJRScoreRetractedPaperYear,SJRQuartileRetractedPaperYear,DemiDecade,MAGAuthorOrderCategorical,MAGFieldName,Field_art,Field_biology,Field_business,Field_chemistry,Field_computerscience,Field_economics,Field_engineering,Field_environmentalscience,Field_geography,Field_geology,Field_history,Field_materialsscience,Field_mathematics,Field_medicine,Field_philosophy,Field_physics,Field_politicalscience,Field_psychology,Field_sociology,MAGRetractionYearAffRankOrdinal,LogMAGCumCitationsAtRetraction,LogMAGCumCollaboratorsAtRetraction,BlogMentionsAtRetraction,F1000MentionsAtRetraction,FacebookMentionsAtRetraction,Google+MentionsAtRetraction,LinkedInMentionsAtRetraction,NewsMentionsAtRetraction,PatentMentionsAtRetraction,Peer reviewMentionsAtRetraction,PinterestMentionsAtRetraction,PolicyMentionsAtRetraction,Q&AMentionsAtRetraction,RedditMentionsAtRetraction,TwitterMentionsAtRetraction,VideoMentionsAtRetraction,WeiboMentionsAtRetraction,WikipediaMentionsAtRetraction,TotalMentionsAtRetraction,AggregateSocialMediaMentionsAtRetraction,AggregateNewsMediaMentionsAtRetraction,AggregateBlogsMentionsAtRetraction,AggregateKnowledgeRepositoriesMentionsAtRetraction,BlogMentionsPreRetraction,F1000MentionsPreRetraction,FacebookMentionsPreRetraction,Google+MentionsPreRetraction,LinkedInMentionsPreRetraction,NewsMentionsPreRetraction,PatentMentionsPreRetraction,Peer reviewMentionsPreRetraction,PinterestMentionsPreRetraction,PolicyMentionsPreRetraction,Q&AMentionsPreRetraction,RedditMentionsPreRetraction,TwitterMentionsPreRetraction,VideoMentionsPreRetraction,WeiboMentionsPreRetraction,WikipediaMentionsPreRetraction,TotalMentionsPreRetraction,AggregateSocialMediaMentionsPreRetraction,AggregateNewsMediaMentionsPreRetraction,AggregateBlogsMentionsPreRetraction,AggregateKnowledgeRepositoriesMentionsPreRetraction,BlogMentionsPostRetraction,F1000MentionsPostRetraction,FacebookMentionsPostRetraction,Google+MentionsPostRetraction,LinkedInMentionsPostRetraction,NewsMentionsPostRetraction,PatentMentionsPostRetraction,Peer reviewMentionsPostRetraction,PinterestMentionsPostRetraction,PolicyMentionsPostRetraction,Q&AMentionsPostRetraction,RedditMentionsPostRetraction,TwitterMentionsPostRetraction,VideoMentionsPostRetraction,WeiboMentionsPostRetraction,WikipediaMentionsPostRetraction,TotalMentionsPostRetraction,AggregateSocialMediaMentionsPostRetraction,AggregateNewsMediaMentionsPostRetraction,AggregateBlogsMentionsPostRetraction,AggregateKnowledgeRepositoriesMentionsPostRetraction,AltmetricScoreAtRetraction,AltmetricScorePreRetraction,AltmetricScorePostRetraction,MAGFirstAffID,FirstAffRank,FirstAffRankCorrected,YearOfAttrition,MAGFirstAffiliationRank
0,2127983451,william r jacobs,2343,1790891469,1997.0,1,1994-07-01,1.332748e+08,NaN,journal of bacteriology,NaN,mistake,5.0,male,0.99,5,2077672427,1976.0,21.0,1997.0,109,1997.0,2441.0,1997.0,266.0,1.299757e+08,1001-,1997.0,86803240.0,0.313531,-19.0,1994-07-01,1994.0,0,-1,journal of bacteriology,journal,NaN,NaN,1996-2000,First or Last or Only Author,biology,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89.0,7.800573,5.587249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.

In [107]:
# Finally let us include the column on retractor

df_reasons = pd.read_csv(OUTDIR+"propagated_reasons_for_paper_matched_sample.csv",
                        usecols=['Record ID', 'RetractorMajority']).drop_duplicates()
df_reasons.columns.tolist()

['Record ID', 'RetractorMajority']

In [108]:
df_final_new = df_final_new.merge(df_reasons, on='Record ID', how='left')

In [114]:
df_final_new[~df_final_new['RetractorMajority'].isna()]\
    [['Record ID','RetractorMajority']].drop_duplicates()

,Record ID,RetractorMajority
9,3631,author
11,2202,author
15,2098,author
34,1325,author
37,33,author
...,...,...
30962,16014,journal
31012,8557,journal
31431,17893,author
31616,1220,author


In [115]:
df_final_new.to_csv(OUTDIR+"/RW_authors_w_confounders_filteredSample_postNHB_BedoorsCorrections_Augmented.csv",
                index=False)

In [116]:
df_final_new['MAGFirstPubYear']

0        1976.0
1        1976.0
2        1976.0
3        1976.0
4        1976.0
          ...  
31732    1991.0
31733    2003.0
31734    2003.0
31735    2010.0
31736    2010.0
Name: MAGFirstPubYear, Length: 31737, dtype: float64

In [24]:
top_100 = [str(rank) for rank in range(101)]
df_final_new[df_final_new.MAGFirstAffiliationRank.isin(top_100)]

,MAGAID,MAGAuthorName,Record ID,MAGPID,RetractionYear,nRetracted,dateobject,JID,CSID,JournalName,...,AggregateBlogsMentionsPostRetraction,AggregateKnowledgeRepositoriesMentionsPostRetraction,AltmetricScoreAtRetraction,AltmetricScorePreRetraction,AltmetricScorePostRetraction,MAGFirstAffID,FirstAffRank,FirstAffRankCorrected,YearOfAttrition,MAGFirstAffiliationRank
0,2127983451,william r jacobs,2343,1790891469,1997.0,1,1994-07-01,133274750.0,NaN,journal of bacteriology,...,0.0,2.0,0.0,3.0,6.00,204465549.0,22.0,22.0,-1.0,22
1,2127983451,william r jacobs,2343,1790891469,1997.0,1,1994-07-01,133274750.0,NaN,journal of bacteriology,...,0.0,2.0,0.0,3.0,6.00,204465549.0,22.0,22.0,-1.0,22
2,2127983451,william r jacobs,2343,1790891469,1997.0,1,1994-07-01,133274750.0,NaN,journal of bacteriology,...,0.0,2.0,0.0,3.0,6.00,204465549.0,22.0,22.0,-1.0,22
3,2127983451,william r jacobs,2343,1790891469,1997.0,1,1994-07-01,133274750.0,NaN,journal of bacteriology,...,0.0,2.0,0.0,3.0,6.00,204465549.0,22.0,22.0,-1.0,22
4,2127983451,william r jacobs,2343,1790891469,1997.0,1,1994-07-01,133274750.0,NaN,journal of bacteriology,...,0.0,2.0,0.0,3.0,6.00,204465549.0,22.0,22.0,-1.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38416,2240430476,ian coldwell,5036,1566674077,2012.0,1,2010-04-01,66322501.0,NaN,sociologia ruralis,...,0.0,0.0,0.0,0.0,0.00,165779595.0,41.0,41.0,2009.0,41
38428,2107017167,wenping liu,6525,1489125205,2015.0,1,2014-01-01,169266912.0,NaN,international journal of advanced robotic systems,...,0.0,0.0,0.0,0.0,0.25,99065089.0,43.0,43.0,2014.0,43
38439,1977276369,ikuko kase,4662,1984811762,2013.0,1,2007-11-01,48908450.0,NaN,genes to cells,...,0.0,0.0,10.0,0.0,0.00,74801974.0,25.0,25.0,2013.0,25
38441,2579513944,longfei cai,282,1988400926,2009.0,1,2008-06-01,91156679.0,NaN,analytical sciences,...,0.0,0.0,0.0,0.0,0.00,76130692.0,70.0,70.0,2008.0,70


In [25]:
df_final_new.MAGFirstAffiliationRank

0             22
1             22
2             22
3             22
4             22
          ...   
38459      1001-
38460    401-500
38461      1001-
38462      1001-
38463      1001-
Name: MAGFirstAffiliationRank, Length: 38464, dtype: object

In [26]:
df_final_new['MAGrootFIDMaxPercent']

0        0.313531
1        0.313531
2        0.313531
3        0.313531
4        0.313531
           ...   
38459    0.166667
38460    0.200000
38461    0.166667
38462    0.142857
38463    0.166667
Name: MAGrootFIDMaxPercent, Length: 38464, dtype: float64

In [27]:
df_final_new['MAGRetractionYearAffYear']

0        1997.0
1        1997.0
2        1997.0
3        1997.0
4        1997.0
          ...  
38459    2013.0
38460    2012.0
38461    2013.0
38462    2014.0
38463    2015.0
Name: MAGRetractionYearAffYear, Length: 38464, dtype: float64

In [28]:
df_final_new['MAGCumPapersYearAtRetraction']

0        1997.0
1        1997.0
2        1997.0
3        1997.0
4        1997.0
          ...  
38459    2013.0
38460    2012.0
38461    2013.0
38462    2014.0
38463    2014.0
Name: MAGCumPapersYearAtRetraction, Length: 38464, dtype: float64

In [29]:
df_final_new['MAGAuthorOrder']

0        5
1        5
2        5
3        5
4        5
        ..
38459    2
38460    3
38461    2
38462    1
38463    2
Name: MAGAuthorOrder, Length: 38464, dtype: int64